In [2]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
import time
import datetime
import os, sys
import pickle
from tqdm import tqdm
from pathlib import Path
from math import ceil, floor
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
from fnmatch import fnmatch
import hist
from hist import Hist

In [3]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
chip_name = "ET2_W36_IP5-17"
chip_figname = f"LowBiasCurrent_HVon100V_C26Removed_Ext10KOhm_{chip_name}"
chip_figtitle= "LowBiasCurrent HVon100V C26Removed Ext10KOhm "+chip_name

fig_outdir = Path('../ETROC-figures')
fig_outdir = fig_outdir / (datetime.date.today().isoformat() + '_Array_Test_Results')
fig_outdir.mkdir(exist_ok=True)
fig_path = str(fig_outdir)

row_list = [14, 14, 14, 14, 15, 15, 15, 15]
col_list = [6, 7, 8, 9, 6, 7, 8, 9]
scan_list = list(zip(col_list, row_list))
print(scan_list)

sigmaScan = [i for i in range(6)]
qinjScan  = [0, 8, 10, 15]

[(6, 14), (7, 14), (8, 14), (9, 14), (6, 15), (7, 15), (8, 15), (9, 15)]


In [5]:
def return_empty_list():
    return {(row,col):{q:{i:0 for i in sigmaScan}for q in qinjScan}for col,row in scan_list}
def return_simple_empty_list():
    return {(row,col):[] for col,row in scan_list}

In [6]:
h = {(row,col):{q:{i:(
        Hist.new.Integer(100, 200, name="CAL", label="CAL [LSB]")
        .Integer(0, 260, name="TOT", label="TOT [LSB]")
        .Integer(0, 520, name="TOA", label="TOA [LSB]")
        .Double()
      )
      for i in sigmaScan}
     for q in qinjScan}
    for col,row in scan_list}
hit_counts = return_empty_list()

## Load the data from the Files

In [7]:
root = '../ETROC-Data'
file_pattern = "*translated*.dat"
for index, row, col in zip(tqdm(range(len(row_list))), row_list, col_list):
    for Qinj in qinjScan:
        for i in sigmaScan:
            path_pattern = f"*2023-06-22_Array_Test_Results/E2_testing_VRef_{chip_figname}_Pixel_C{col}_R{row}_BLplus{i}sigma_Qinj_{Qinj}_Threshold*"
            file_list = []
            for path, subdirs, files in os.walk(root):
                if not fnmatch(path, path_pattern): continue
                for name in files:
                    pass
                    if fnmatch(name, file_pattern):
                        file_list.append(os.path.join(path, name))
                        # print(file_list[-1])
            total_files = len(file_list)
            for file_index, file_name in enumerate(file_list):
                # print(f"{file_index+1}/{total_files}")
                with open(file_name) as infile:
                    for line in infile:
                        text_list = line.split()
                        if text_list[2]!="DATA": continue
                        c_running = int(text_list[6])
                        r_running = int(text_list[8])
                        if(c_running!=col or r_running!=row): continue
                        TOA = int(text_list[10])
                        TOT = int(text_list[12])
                        CAL = int(text_list[14])
                        h[(r_running,c_running)][Qinj][i].fill(CAL,TOT,TOA)
                        hit_counts[(r_running,c_running)][Qinj][i] += 1 

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [01:27<00:00, 10.96s/it]


## Make Plots

In [9]:
for Qinj in qinjScan:
    fig = plt.figure(dpi=300, figsize=(20,10))
    u_cl = np.sort(np.unique(col_list))
    u_rl = np.sort(np.unique(row_list))
    gs = fig.add_gridspec(len(u_rl),len(u_cl))
    for ri,row in enumerate(u_rl):
        for ci,col in enumerate(u_cl):
            ax0 = fig.add_subplot(gs[len(u_rl)-ri-1,len(u_cl)-ci-1])
            for i in sigmaScan:
                h[(row,col)][Qinj][i].project("CAL")[100j:200j].plot1d(ax=ax0, lw=1, label=fr"$\mu$+{i}$\sigma$", density=True)
            plt.title(chip_figtitle+f", CAL [LSB], Pixel [{row},{col}], Qinj {Qinj}fC", size=6)
            ax0.set_xticks(ax0.get_xticks()[::2],ax0.get_xticklabels()[::2], rotation='vertical', size=5)
            plt.legend()
    plt.tight_layout()
    plt.savefig(str(fig_outdir)+f"/noiseAnalysis_Qinj_{Qinj}_CAL_"+chip_figname+"_"+datetime.date.today().isoformat()+".png")
    plt.close()

/Users/murtazas-local/.pyenv/versions/my_3.9.16/lib/python3.9/site-packages/mplhep/utils.py:262: RuntimeWarning: divide by zero encountered in double_scalars
  self.flat_scale(1 / np.sum(np.diff(self.edges) * self.values))
/Users/murtazas-local/.pyenv/versions/my_3.9.16/lib/python3.9/site-packages/mplhep/utils.py:197: RuntimeWarning: All sumw are zero!  Cannot compute meaningful error bars
  return np.abs(method_fcn(self.values, variances) - self.values)
/Users/murtazas-local/.pyenv/versions/my_3.9.16/lib/python3.9/site-packages/mplhep/utils.py:242: RuntimeWarning: invalid value encountered in multiply
  self.values *= scale
/Users/murtazas-local/.pyenv/versions/my_3.9.16/lib/python3.9/site-packages/mplhep/utils.py:243: RuntimeWarning: invalid value encountered in multiply
  self.yerr_lo *= scale
/Users/murtazas-local/.pyenv/versions/my_3.9.16/lib/python3.9/site-packages/mplhep/utils.py:244: RuntimeWarning: invalid value encountered in multiply
  self.yerr_hi *= scale
/Users/murtazas-l

In [12]:
for Qinj in qinjScan:
    fig = plt.figure(dpi=300, figsize=(20,10))
    u_cl = np.sort(np.unique(col_list))
    u_rl = np.sort(np.unique(row_list))
    gs = fig.add_gridspec(len(u_rl),len(u_cl))
    for ri,row in enumerate(u_rl):
        for ci,col in enumerate(u_cl):
            ax0 = fig.add_subplot(gs[len(u_rl)-ri-1,len(u_cl)-ci-1])
            for i in sigmaScan:
                h[(row,col)][Qinj][i].project("TOT").plot1d(ax=ax0, lw=1, label=fr"$\mu$+{i}$\sigma$")
            plt.title(chip_figtitle+f", TOT [LSB], Pixel [{row},{col}], Qinj {Qinj}fC", size=6)
            ax0.set_xticks(ax0.get_xticks()[::4],ax0.get_xticklabels()[::4], rotation='vertical', size=5)
            plt.legend()
    plt.tight_layout()
    plt.savefig(str(fig_outdir)+f"/noiseAnalysis_Qinj_{Qinj}_TOT_"+chip_figname+"_"+datetime.date.today().isoformat()+".png")
    plt.close()

/Users/murtazas-local/.pyenv/versions/my_3.9.16/lib/python3.9/site-packages/mplhep/utils.py:197: RuntimeWarning: All sumw are zero!  Cannot compute meaningful error bars
  return np.abs(method_fcn(self.values, variances) - self.values)
/Users/murtazas-local/.pyenv/versions/my_3.9.16/lib/python3.9/site-packages/mplhep/utils.py:197: RuntimeWarning: All sumw are zero!  Cannot compute meaningful error bars
  return np.abs(method_fcn(self.values, variances) - self.values)
/Users/murtazas-local/.pyenv/versions/my_3.9.16/lib/python3.9/site-packages/mplhep/utils.py:197: RuntimeWarning: All sumw are zero!  Cannot compute meaningful error bars
  return np.abs(method_fcn(self.values, variances) - self.values)
/Users/murtazas-local/.pyenv/versions/my_3.9.16/lib/python3.9/site-packages/mplhep/utils.py:197: RuntimeWarning: All sumw are zero!  Cannot compute meaningful error bars
  return np.abs(method_fcn(self.values, variances) - self.values)
/Users/murtazas-local/.pyenv/versions/my_3.9.16/lib/pyth

In [13]:
for Qinj in qinjScan:
    fig = plt.figure(dpi=300, figsize=(20,10))
    u_cl = np.sort(np.unique(col_list))
    u_rl = np.sort(np.unique(row_list))
    gs = fig.add_gridspec(len(u_rl),len(u_cl))
    for ri,row in enumerate(u_rl):
        for ci,col in enumerate(u_cl):
            ax0 = fig.add_subplot(gs[len(u_rl)-ri-1,len(u_cl)-ci-1])
            for i in sigmaScan:
                h[(row,col)][Qinj][i].project("TOA").plot1d(ax=ax0, lw=1, label=fr"$\mu$+{i}$\sigma$")
            plt.title(chip_figtitle+f", TOA [LSB], Pixel [{row},{col}], Qinj {Qinj}fC", size=6)
            ax0.set_xticks(ax0.get_xticks()[::7],ax0.get_xticklabels()[::7], rotation='vertical', size=5)
            plt.legend()
    plt.tight_layout()
    plt.savefig(str(fig_outdir)+f"/noiseAnalysis_Qinj_{Qinj}_TOA_"+chip_figname+"_"+datetime.date.today().isoformat()+".png")
    plt.close()

/Users/murtazas-local/.pyenv/versions/my_3.9.16/lib/python3.9/site-packages/mplhep/utils.py:197: RuntimeWarning: All sumw are zero!  Cannot compute meaningful error bars
  return np.abs(method_fcn(self.values, variances) - self.values)
/Users/murtazas-local/.pyenv/versions/my_3.9.16/lib/python3.9/site-packages/mplhep/utils.py:197: RuntimeWarning: All sumw are zero!  Cannot compute meaningful error bars
  return np.abs(method_fcn(self.values, variances) - self.values)
/Users/murtazas-local/.pyenv/versions/my_3.9.16/lib/python3.9/site-packages/mplhep/utils.py:197: RuntimeWarning: All sumw are zero!  Cannot compute meaningful error bars
  return np.abs(method_fcn(self.values, variances) - self.values)
/Users/murtazas-local/.pyenv/versions/my_3.9.16/lib/python3.9/site-packages/mplhep/utils.py:197: RuntimeWarning: All sumw are zero!  Cannot compute meaningful error bars
  return np.abs(method_fcn(self.values, variances) - self.values)
/Users/murtazas-local/.pyenv/versions/my_3.9.16/lib/pyth

In [14]:
for Qinj in qinjScan:
    for row,col in zip(row_list, col_list):
        fig = plt.figure(dpi=300, figsize=(15,10))
        gs = fig.add_gridspec(2,3)
        for i in sigmaScan:
            ax = fig.add_subplot(gs[i//3,i%3])
            h[(row,col)][Qinj][i].project("TOA", "TOT").plot2d(
                ax=ax,
                cmap="coolwarm"
            )
            ax.set_xticks(ax.get_xticks()[::7],ax.get_xticklabels()[::7], rotation='vertical', size=5)
            ax.set_yticks(ax.get_yticks()[::4],ax.get_yticklabels()[::4], size=5)
            ax.set_title(chip_figtitle+fr", TOA-TOT, Pixel [{row},{col}], Qinj {Qinj}fC, $\mu$+{i}$\sigma$", size=6)
        plt.tight_layout()
        plt.savefig(str(fig_outdir)+f"/noiseAnalysis_Qinj_{Qinj}_TOA_vs_TOT_Pixel_{row}_{col}_"+chip_figname+"_"+datetime.date.today().isoformat()+".png")
        plt.close()
